# Preprocessing Stage  - Part 1 #

Below is the code for opening the historical sea surface temperature model data, creating the MMM threshold and then creating an array of HotSpot values. This will be done by using the xarray python package and its capabilities for assessing multidimensional data. This is the first step in the data processing stage before we are able to calcuate the MHW variables. The code will show the steps for the model data as an example, but with very small changes this same script could be applied to observed data, such as the coraltemp data we will be using later.

This notebook was given to me already created, and all I had to do was run it and see if I could save the results. Below I will be showing the original script as well as some alternative ways of calculating the results with the xarray package that I have found. The parts that are from the original script will be labeled "ORIGINAL" and the others will be labeled "ALTERNATIVE".

In [ ]:
#importing all the packages that we will be needing for the current step
import netCDF4 as nc
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Opening and Selecting data ###

ORIGINAL

In [ ]:
# read in daily sst from historical simulation (1985-2014) and future simulation of ssp245(2015-2019)
sst_8089 = xr.open_dataset('tos_Oday_GFDL-ESM4_historical_r1i1p1f1_gr_19800101-19891231.nc')  # another way by putting URL: https://furtherinfo.es-doc.org/CMIP6.NCAR.CESM2.ssp245.none.r11i1p1f1
sst_8589 = sst_8089.sel(time=slice('1985', '1989')).load()

sst_9099 = xr.open_dataset('tos_Oday_GFDL-ESM4_historical_r1i1p1f1_gr_19900101-19991231.nc')
sst_0009 = xr.open_dataset('tos_Oday_GFDL-ESM4_historical_r1i1p1f1_gr_20000101-20091231.nc')
sst_1014 = xr.open_dataset('tos_Oday_GFDL-ESM4_historical_r1i1p1f1_gr_20100101-20141231.nc')

# concatenate the sst datasets along the deimension of time to get a single variable with daily sst from 1985 to 2014
sst_8514 = xr.concat([sst_8589, sst_9099, sst_0009, sst_1014], dim='time')

ALTERNATIVE

In [ ]:
#by using xr.open_mfdataset and adding the conditions below we are able to achieve the same results as the above cell
sst_8514 = delayed(xr.open_mfdataset('tos_Oday_GFDL-ESM4_historical_r1i1p1f1_gr_*.nc', 
                             combine = 'nested', concat_dim = 'time')).tos.sel(time=slice('1985', '2014'))

### Creating MMM threshold ###

ORIGINAL

In [ ]:
## calculate the monthly MMM
# method1: group daily dataset by month 
gb = sst_8514.groupby('time.month')

# calculate monthly mean sst in each grid for each year
sst_8514_mm = gb.mean(dim='time')

#select Maximum Monthly mean
sst_8514_MMM = sst_8514_mm.max(axis=0)

# write out MMM from 1985 to 2014
fn = 'MMM8514_GFDLESM4py.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

#time = ds.createDimension('time', None)  #Unlimited dimensions is specified by 'None', which allows for the time variable to grow.
lat = ds.createDimension('lat', 180)
lon = ds.createDimension('lon', 360)

lats = ds.createVariable('lat', 'f4', ('lat',))
lons = ds.createVariable('lon', 'f4', ('lon',))
MMM = ds.createVariable('MMM_8514', 'f4', ('lat', 'lon'))
MMM.units = 'degree celcius'

# read in latitude and longitude variables
lats[:] = sst_8589.lat
lons[:] = sst_8589.lon
MMM[:,:] = sst_8514_MMM.tos

ds.close()

ALTERNATIVE

In [ ]:
#same process of grouping and finding the mean and then the max of the mean but then to create the NETCDF file there is a much simpler built in function in Xarray 
gb_mm = sst_8514.groupby('time.month').mean()
sst_8514_MMM = gb_mm.max(dim = 'time')
sst_8514_MMM.to_netcdf('MMM8514_GFDLESM4py.nc')

### Calculating HotSpot ###

ORIGINAL

In [ ]:
# only store the postive SST anomalies

HS = np.zeros((180, 360, 10950))  # HS.shape finds the size of the data array which is 180, 360, 10950

for t in list(range(10950)):
    HS[:, :, t] = sst_8514[:, :, t] - sst_8514_MMM[:, :]
    for i in list(range(180)):
        for j in list(range(360)):
            if HS[i, j, t] < 0:
                HS[i, j, t] =  0 
                
# create a dataarray of HS with coordinates 
HS_MMM = xr.DataArray(HS, coords=[sst_8514.time, sst_8514.lat, sst_8514.lon], dims=[ "time", "lat", "lon"])
HS_MMM.shape

# write out MMM from 1985 to 2014
fn = 'DHS4_GFDLESM4py_1.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

#time = ds.createDimension('time', None)  #Unlimited dimensions is specified by 'None', which allows for the time variable to grow.
latID = ds.createDimension('latID', 180)
lonID = ds.createDimension('lonID', 360)
tID = ds.createDimension('tID', 10950)

lats = ds.createVariable('lat', 'f4', ('latID',))
lons = ds.createVariable('lon', 'f4', ('lonID',))
ts = ds.createVariable('time', 'f8', ('tID',))
HS = ds.createVariable('HS_MMM', 'f4', ('latID', 'lonID', 'tID'))
HS.units = 'degree_celcius'
lats.units = 'degree'
lons.units = 'degree'
ts.units = 'days since 1850-01-01 00:00:00'

# read in latitude and longitude variables
lats[:] = sst_8589.lat
lons[:] = sst_8589.lon
ts[:] = times   
HS[:,:,:] = HS_MMM[:, :, :]

ds.close()

ALTERNATIVE

The alternative for this part consists of using some built in Xarray functions like .where instead of the for loop.

In [ ]:
#here we are calculating the difference between the original values of sea surface temperature and the MMM
HS = sst_8514 - sst_8514_MMM

#then we are creating a dataArray where if HS is less than 0 the value will be set to 0, and if not the value will be set to the HS value,
#this is so that only positive HS values are taken into account since we are looking at only the warm seasons
HS_MMM = xr.where(HS < 0, 0, HS)

#the last step is just to create a NetCDF file the same way we did above for MMM
HS_MMM.to_netcdf('DHS4_GFDLESM4py_1.nc')

# Preprocessing Stage - Part 2 #

The second part of the preprocessing stage was creating a mask representing the month where the lowest sea surface temperature occured for each gridcell. This would be later used to generate the time series for the MHW variable calculations which we call the Heat Stress Year. 

### Opening and Selecting Data ###
We have two options here: either start from scratch so we would open up the model/observed data again as we did in the Opening and Selecting Data in Part 1. Or we can just continue in the same notbook if our memory allows it and just use the variables already created above. I will be doing the latter below and just accesing the variables that were created above to continue the preprocessing stage.

### Calculating month of Lowest SST for each gridcell ###
The .idxmin function built in to Xarray and Numpy gives us the index that the lowest value occured in for the dimension that we specify in the parenthesis, so in this case month. If the lowest sst occured in march for example the result of idxmin for that gridcell would be 3.

In [ ]:
#using the variable gb_mm which is the groupby 'time.month' mean result
min_month = gb_mm.idxmin('month') 

### Creating Mask of Months ###
Here we can use the .where function again to create our masked DataArray. What the cell below is saying is for all the values that are not nan, we apply another where function that gives the cells with min_month between 1 and 5 a value of 10 and all the cells with min_month between 6 and 12 a value of 4, all the calls that had a value of nan will continue having a nan value

In [ ]:
masked = xr.where(min_month.notnull(), (xr.where(min_month < 6, 10, 4)), np.nan)

In [ ]:
#if we want to visualize the masked array we can plot it 
masked.plot()

### Saving ###

In [ ]:
#we use the same approach we used for the two previous NetCDF files above
masked.to_netcdf('monthly_mask.nc')

# Preprocessing Stage - Part 3 #

The third and last part of the Preprocessing stage before we can start the analysis is to add a new HSY time series to our HotSpot data. In this step we need to be careful because in a specific step it will be very different depending on whether we are working with nonleap year data (like the model data) or with leap year data (like the coraltemp data). 

Basically, what the code is doing here is first creating two separate datasets out of the Hot Spot data, one that starts in March and ends in February of the following year and the other that starts in September and ends in August of the following year. Then for the March to February time series we select only the values that correspond to grid cells with the coldest month between January and May (HS10_mask), and for the September to August time series we select only the values that correspond to grid cells with the coldest month occurring between June and December (HS4_mask). Masked_array is a previous dataset I had created where I found the coldest month for each grid cell and assigned it either a 10 or a 4. This represents how many months are included from the first year calculated. The next step is where we get a little creative, we create a daily custom date range starting January first of 1985 and then we just assign this date range to the two datasets we had previously created as the time coordinate. So, for example, the first data point for the March-February dataset actually occurred March 1, 1985, but it will be labeled as January 1, 1985, because it is the first day of our defined Heat Stress Year. Then all we have to do is combine the two datasets (Mar – Feb) and (Sep – Aug) and now we have one dataset with labels corresponding to our HSY definition but data corresponding to the grid cells specific warm season conditions. 


### Opening and Selecting Data ###

Once again this step will be different depending on if you are starting a new notebook or if we are just continuing with the variables that have already been created abouve. If we were starting a new notebook we would need to open two datasets. The first is the HS data we created in part 1 and the second would be the monthly_mask NetCDf file we created in part 2.

For the example below we will be continuing with the variable names that were deifned above. So HS_MMM will be the HotSpot data and masked will be the monthly_mask data.

In [ ]:
#here we are creating two new dataArrays both from the HS_MMM dataArray but 
#one is for the time period of march 1 1985 to february 28 2014, and the other is for the time period of september 1 1985 to august 31 2014
#CAUTION: this is a step that will change depending on if we have leap years or not. If there is a leap year and the last year we are including for HS10 is a leap year we would have change it from february 28 to 29.
#The example code below is for model data so there are no leap years
HS10 = HS_MMM.sel(time=slice('1985-03-01','2014-02-28'))
HS4 = HS_MMM.sel(time=slice('1985-09-01','2014-08-31'))

In [ ]:
#In this cell we are saying for HS10_mask to only include data in HS10 where mask is 10 and everything else is set to 0
#The same is happening for HS4_mask, it will only include data in HS4 where mask is 4 and everything else is set to 0
HS10_mask = HS10.where(mask == 10).to_dataset()
HS4_mask = HS4.where(mask == 4).to_dataset()

In [ ]:
#In this cell we are just creating variables that represent the HS data for HS10_mask and HS4_mask (we will use this later)
hs10 = HS10_mask.HS_MMM
hs4 = HS4_mask.HS_MMM

### Creating a Custom Calendar ###
The second step is to create a calendar that has the same number of days as our data. This will be different depending on whether or not we have leap years in our data. In the cell below I have written out the two options of calendars, before running the cell we should comment out the one that will not be used. 

In [ ]:
#change to appropriate calendar type for the data being used, this is an example for data with no leap years
calendar = xr.cftime_range(start="1985-01-01", periods=365 *29, freq="D", calendar="noleap")
#this is how to create the calender to include leap years
calendar = pd.date_range("1985-01-01", periods=12418)

In [ ]:
#In this cell we are creating two DataVariables with the time coordinate labels from our calendar but the time values from the HS10_mask and HS4_mask
ds4t = xr.Dataset({"ds4": ("time",calendar), "time": HS4_mask.time}).set_coords('ds4')
ds10t = xr.Dataset({"ds10": ("time", calendar), "time": HS10_mask.time}).set_coords('ds10')

### Creating New HotSpot DataArray ###
Now the only thing left to do is to create two new DataArrays that have the same lat, lon and data as the DataArryas we created in the data opening and processing stage (this is the same HS data just for a specifically selected time period based on the month_mask data), and the time cordinate will be the new calendar we created

In [ ]:
HS_10t = xr.DataArray(hs10, coords=[ds10t.ds10, HS10_mask.lat, HS10_mask.lon], dims=["time", "lat", "lon"]).to_dataset()
HS_4t = xr.DataArray(hs4, coords=[ds4t.ds4, HS4_mask.lat, HS4_mask.lon], dims=["time", "lat", "lon"]).to_dataset()

If we plot the two DataArrays above we can see that they each just cover the parts of the world that correspond to either 10 or 4 in the month_mask data (more simply they each just cover half the world) so we need to combine them to get data for the whole world for the time period of 30 years.

In [ ]:
#here we can use the .combine_first function from xarray to accomplish combining the two "halves" of the world into one
HSY_daily_t = HS_4t.combine_first(HS_10t)

### Saving the Data ###
The last step is just to save this new HotSpot dataset that has the correct time series data for our analysis

In [ ]:
HSY_daily_t.to_netcdf('HS_HSY_daily.nc')